In [80]:
import brainsss
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from sklearn.cluster import AgglomerativeClustering
import scipy
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from scipy.cluster import hierarchy
from scipy.signal import butter, filtfilt, freqz
from scipy import signal
import matplotlib as mpl
from matplotlib.pyplot import cm
import random
from scipy.stats import sem, zscore
import time
import h5py
import ants
import nibabel as nib
import matplotlib
from scipy.ndimage import gaussian_filter1d
import pickle
from skimage import io
import glob

In [81]:
fly_num = 'fly_250'
func_path = f'/oak/stanford/groups/trc/data/Ilana/2P/data/{fly_num}/'
later_dir = '/oak/stanford/groups/trc/data/Ilana/2P/data/later/'


In [82]:
###########################
### PREP VISUAL STIMULI ###
###########################

vision_path = os.path.join(func_path,'func_0', 'visual')

### Load Photodiode ###
t, ft_triggers, pd1, pd2 = brainsss.load_photodiode(vision_path)
stimulus_start_times = brainsss.extract_stim_times_from_pd(pd1, t)

# *100 puts in units of 10ms, which will match fictrac
st_10ms = [int(stimulus_start_times[i]*100) for i in range(len(stimulus_start_times))]

# get 1ms version to match neural timestamps
st_ms= [i*10 for i in st_10ms]
starts_loom = st_10ms

loading photodiode data... done


In [83]:
####################
### Prep Fictrac ###
####################

fictrac_path = os.path.join(func_path, 'func_0', 'fictrac')
fictrac_raw = brainsss.load_fictrac(fictrac_path)

fps = 100
resolution = 10 #desired resolution in ms
expt_len = fictrac_raw.shape[0]/fps*1000
behaviors = ['dRotLabY', 'dRotLabZ', 'dRotLabX']
fictrac = {}
for behavior in behaviors:
    if behavior == 'dRotLabY': short = 'Y'
    elif behavior == 'dRotLabZ': short = 'Z'
    elif behavior == 'dRotLabX': short = 'X'
    fictrac[short] = brainsss.smooth_and_interp_fictrac(fictrac_raw, fps, resolution, expt_len, behavior)
fictrac_timestamps = np.arange(0,expt_len,resolution)

In [84]:
###########################################
### Extract Stimulus Triggered Behavior ###
###########################################

pre_window = 200
post_window = 300
# avg_around = 20
stim_time = 100
# window = np.arange(-pre_window,post_window)

behavior_traces = {}
mean_trace = {}
sem_trace = {}
behavior_traces,mean_trace,sem_trace = brainsss.extract_traces(fictrac['Y'], starts_loom, pre_window, post_window)
# behavior_traces_QC,mean_trace_QC,sem_trace_QC = extract_traces(fictrac['Y'], null_times, pre_window, post_window)


In [85]:
starts_loom_ms = [n*10 for n in starts_loom]

In [86]:
n_clusters = 2000
cluster_dir = os.path.join(func_path, 'func_0','clustering')

ch_num = '2'#'1'
label_name =''
signal_name =''

for x in sorted(os.listdir(cluster_dir)):
    if 'label' in x and ch_num in x:
        label_name = x
    if 'signals' in x and ch_num in x:
        signal_name = x
    elif 'signals' in x:
        signal_name = x
    elif 'label' in x:
        label_name = x

print(label_name)
print(signal_name)
        
load_file = os.path.join(cluster_dir, label_name)
cluster_labels = np.load(load_file)

load_file = os.path.join(cluster_dir, signal_name)
all_signals = np.load(load_file)

cluster_labels_2.npy
cluster_signals_2.npy


In [87]:
timestamps = brainsss.load_timestamps(os.path.join(func_path,'func_0', 'imaging'))

Trying to load timestamp data from hdf5 file.
Success.


In [88]:
bin_start = -500; bin_end = 2000; bin_size = 100 #ms
neural_bins = np.arange(bin_start,bin_end,bin_size)

In [89]:
%%time
data_dict = {}
data_dict['signals'] = all_signals
data_dict['timestamps'] = timestamps
data_dict['event_times'] = starts_loom_ms
data_dict['bins'] = neural_bins
data_dict['behavior'] = behavior_traces
data_dict['cluster_labels'] = cluster_labels


later_path = os.path.join(later_dir, fly_num+'_dic.pkl')

with open(later_path, 'wb') as file:
    pickle.dump(data_dict, file)

CPU times: user 663 ms, sys: 2.56 s, total: 3.22 s
Wall time: 4.84 s


In [90]:
for x in sorted(os.listdir(later_dir)):
    if 'dic' in x:
        print(x)

fly_208_dic.pkl
fly_209_dic.pkl
fly_210_dic.pkl
fly_217_dic.pkl
fly_218_dic.pkl
fly_226_dic.pkl
fly_227_dic.pkl
fly_228_dic.pkl
fly_233_dic.pkl
fly_234_dic.pkl
fly_239_dic.pkl
fly_240_dic.pkl
fly_241_dic.pkl
fly_242_dic.pkl
fly_249_dic.pkl
fly_250_dic.pkl
total_dict.pkl
